# 深度强化学习 DQN 实验

实验采用 OpenAI Gymnasium 的 Frozen Lake 环境

![frozen_lake](imgs/frozen_lake.gif)

In [ ]:
import collections
import random
import time
import torch
import torch.nn as nn
import torch.optim as optim
import gymnasium as gym
import matplotlib.pyplot as plt

In [ ]:
# 训练设备
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f'Using device: {device}')

## 环境

环境为一个 4x4 的网格世界，Agent 可以往上下左右四个方向移动。Agent 从起点出发，需要避开冰湖到达礼物的位置。

状态空间为：$\mathbf{S} = \{0,1,...,15\}$

动作空间为：$\mathbf{A} = \{0,1,2,3\}$

In [ ]:
# 网格世界各网格可能的状态 token
BLANK = 0
LAKE = 1
GIFT = 2
PLAYER = 3

# 网格世界地图
grid = torch.tensor([
    [BLANK, BLANK, BLANK, BLANK],
    [BLANK, LAKE, BLANK, LAKE],
    [BLANK, BLANK, BLANK, LAKE],
    [LAKE, BLANK, BLANK, GIFT]
], dtype=torch.long)

# 将玩家 token 放置在地图上
def player_on_grid(state):
    batch_size = state.size(0)
    ret = grid.unsqueeze(0).repeat(batch_size, 1, 1).to(state.device)
    indices = torch.cat((state.unsqueeze(-1) // 4, state.unsqueeze(-1) % 4), dim=-1)
    for i in range(batch_size):
        ret[i, indices[i,0], indices[i,1]] = PLAYER
    return ret

## 模型

<br>

#### 模型结构

模型采用一个嵌入层和三层卷积网络提取特征，然后通过两个线性层估计Q值。

<br>

#### 模型训练

训练过程分为两部分，首先需要从环境中获取训练样本，称为经验回放，存在经验回放缓冲器中，然后根据经验回放训练模型。其中获取经验回放需要使用 Epsilon-Greedy 算法。

<br>

**从环境中采样过程如下**：

首先使用 Epsilon-Greedy 获取动作：
$$
x \sim \text{Uniform}[0,1)
$$

$$
\begin{cases}
a = \underset{a}{\text{argmax}} Q(s,a), & x \gt \epsilon \\
a \sim \text{Uniform}(\mathbf{A}), & x \le \epsilon
\end{cases}
$$

然后将动作应用到环境，可以获取一个五元组 $(s_t,a_t,r_t,s_{t+1},\text{done})$ 。将这个元组存储到经验回放缓冲器，直到数量达到缓冲器的最大数量。

<br>

**训练过程如下**：

从经验回放缓冲器中随机采样一批数据，并计算 loss：
$$
\text{target} = r_t + \gamma \underset{a_{t+1}}{\text{max}} Q(s_{t+1},a_{t+1})
$$
其中 $\gamma$ 是折扣因子，

$$
\text{loss} = \text{MSELoss}(Q(s_t,a_t), \text{target})
$$

然后就可以对 loss 进行反向传播啦。

In [ ]:
# 深度Q网络模型
class DQN(nn.Module):
    def __init__(self, state_size=(4, 4), action_size=4, grid_states=4, d_model=16, lr=1e-3):
        super().__init__()
        # 模型结构参数
        self.state_size = state_size
        self.action_size = action_size
        self.grid_states = grid_states
        self.d_model = d_model
        
        # 深度网络
        self.emb = nn.Embedding(self.grid_states, self.d_model)
        self.featnet = nn.Sequential(
            nn.Conv2d(d_model, 2*d_model, kernel_size=3, stride=1, padding=1),          # (4,4,16) -> (4,4,32)
            nn.ReLU(),
            nn.Conv2d(2*d_model, 4*d_model, kernel_size=3, stride=1, padding=1),        # (4,4,32) -> (4,4,64)
            nn.ReLU(),
            nn.Conv2d(4*d_model, 8*d_model, kernel_size=3, stride=1),                   # (4,4,64) -> (2,2,128)
            nn.ReLU()
        )
        self.vnet = nn.Sequential(
            nn.Linear(self._feat_size(), self._feat_size() // 8),                        # (512) -> (64)
            nn.ReLU(),
            nn.Linear(self._feat_size() // 8, self.action_size)                          # (64) -> (4)
        )
        
        # 采用均方差损失函数
        self.criterion = nn.MSELoss()

        # 迭代相关的数据
        self.iterations = 0
        self.optimizer = optim.Adam(self.parameters(), lr=lr)
        
    def _feat_size(self):
        return 8 * (self.state_size[0] - 2) * (self.state_size[1] - 2) * self.d_model
        
    # 前向过程
    def forward(self, x: torch.Tensor):
        batch_size = x.size(0)
        x = player_on_grid(x)
        x = self.emb(x)
        x = x.permute(0, 3, 1, 2)
        x = self.featnet(x)
        x = x.reshape(batch_size, -1)
        x = self.vnet(x)
        return x

    # 采样
    def act(self, x: int, eps=0.1):
        # Epsilon-Greedy
        if random.random() > eps:
            with torch.no_grad():
                out = self.forward(torch.tensor([x], device=device))
                return torch.argmax(out, dim=-1).squeeze().cpu().item()
        else:
            return random.randint(0, self.action_size-1)

    # 训练
    def update(self, sample, gamma=0.9):
        state, action, reward, next_state, done = sample
        # tuple 转 tensor
        state = torch.tensor(state, device=device)
        action = torch.tensor(action, device=device)
        reward = torch.tensor(reward, device=device)
        next_state = torch.tensor(next_state, device=device)
        done = torch.tensor(done, dtype=torch.bool, device=device)
        
        # 训练主要逻辑
        self.optimizer.zero_grad()
        max_next_q = torch.max(self.forward(next_state), dim=1)[0] * (~done)
        target = reward + gamma * max_next_q
        q = self.forward(state).gather(1, action[:,None])[:,0]
        loss = self.criterion(q, target)
        loss.backward()
        self.optimizer.step()
        self.iterations += 1
        return loss.item()

In [ ]:
# 经验回放缓冲器
class ExpReplayBuffer:
    def __init__(self, max_len=1024):
        self.max_len = max_len
        self.buffer = collections.deque(maxlen=max_len)     # 使用队列存储
    
    def clear(self):
        self.buffer.clear()
        
    def is_full(self):
        return len(self.buffer) == self.max_len
    
    def append(self, sample):
        self.buffer.append(sample)
    
    # 随机取出一定样本用于模型训练
    def sample(self, n=256):
        zipped = random.sample(list(self.buffer), n)
        return list(zip(*zipped))

In [ ]:
# 从环境中采样数据用于训练模型
def sample_from_env(env: gym.Env, net: nn.Module, buffer: ExpReplayBuffer):
    net.eval()
    obs = 0
    env_running = False
    buffer.clear()
    while not buffer.is_full():
        if not env_running:
            obs, info = env.reset()
            env_running = True
        action = net.act(obs)
        obs_next, reward, terminated, truncated, info = env.step(action)
        env_running = not (terminated or truncated)
        buffer.append((obs, action, reward, obs_next, terminated))
        obs = obs_next
    env.close()

# 训练模型
def train_model(net: nn.Module, buffer: ExpReplayBuffer, batch_size=256):
    samples = buffer.sample(batch_size)
    net.train()
    return net.update(samples, gamma=0.9)

In [ ]:
# 推理，无图形化渲染，用于采集训练时的模型拟合情况
def infer(env: gym.Env, net: nn.Module):
    net.eval()
    obs, info = env.reset()
    episode_over = False
    steps = 0
    reward_cnt = 0
    while not episode_over:
        action = net.act(obs, eps=0.0)
        obs_next, reward, terminated, truncated, info = env.step(action)
        episode_over = terminated or truncated
        obs = obs_next
        steps += 1
        reward_cnt += reward
    env.close()
    return reward_cnt, steps

# 推理，图形化渲染，用于直观地观察模型的运动
def infer_human_render(env: gym.Env, net: nn.Module):
    net.eval()
    obs, info = env.reset()
    env.render()
    episode_over = False
    steps = 0
    reward_cnt = 0

    time.sleep(2)

    while not episode_over:
        action = net.act(obs, eps=0.0)
        obs_next, reward, terminated, truncated, info = env.step(action)
        env.render()

        time.sleep(2)

        episode_over = terminated or truncated
        obs = obs_next
        steps += 1
        reward_cnt += reward
    env.close()
    if reward_cnt == 0:
        print(f'模型未到达终点，共计{steps}步')
    else:
        print(f'模型到达终点，共计{steps}步')

In [ ]:
# 保存模型
def save_model(net: DQN):
    state_dict = {
        'iterations': net.iterations,
        'model': net.state_dict(),
        'optimizer': net.optimizer.state_dict(),
    }
    save_path = f'./models/dqn_{net.iterations}.pth'
    torch.save(state_dict, save_path)
    print(f'Model saved at {save_path}.')

# 加载模型
def load_model(net: DQN, path: str):
    state_dict = torch.load(path)
    net.iterations = state_dict['iterations']
    net.load_state_dict(state_dict['model'])
    net.optimizer.load_state_dict(state_dict['optimizer'])
    print(f'Model loaded from {path}.')

In [ ]:
env = gym.make('FrozenLake-v1')
net = DQN()
net = net.to(device)
buffer = ExpReplayBuffer(max_len=1024)

loss_rec = []
reward_rec = []
steps_rec = []

for i in range(1500):
    sample_from_env(env, net, buffer)
    loss = train_model(net, buffer, batch_size=256)
    reward, steps = infer(env, net)
    loss_rec.append(loss)
    reward_rec.append(reward)
    steps_rec.append(steps)
    print(f'Iteration {net.iterations} finished with loss {loss}')

In [ ]:
save_model(net)

In [ ]:
plt.subplot(3, 1, 1)
plt.plot(loss_rec)
plt.subplot(3, 1, 2)
plt.plot(reward_rec)
plt.subplot(3, 1, 3)
plt.plot(steps_rec)
plt.show()